In [2]:
def optimize(course_info, room_capacities, professor_info, output_info):
    
    ## converting input values as strings to be used in pandas commands
    course_data = str(course_info)
    room_data = str(course_info)
    prof_data = str(professor_info)
    output_info = str(output_info)
    
    ## importing necessary packages
    import pandas as pd
    import re #for string matching elements
    from gurobipy import Model, GRB # for optimization
    from itertools import product # for reading the optimized values 
    
    ## reading in the course and timeslot data, along with the final schedule structure
    classes = pd.read_excel(course_data,sheet_name='Timesheet',index_col=0)
    
    ## pulling course names, section numbers, and core course indications from given data
    core = pd.read_excel(course_data, sheet_name='Core')
    core.columns = ['course', 'Section', 'type']
    core.iloc[:, 1] = core.iloc[:,1].astype(str) # for equal type comparisons to other data
    
    ## pulling demand per section from historical data
    courses = pd.read_excel(course_data,sheet_name='Demand')
    courses.iloc[:, 0] = courses.iloc[:,0].astype(str) # for equal type comparisons to other data
    
    ## pulling classroom capacity from historical data
    capacity = pd.read_csv(room_data)
    
    ## pulling data to match professors to section number
    raw = pd.read_csv(prof_data)
    raw = raw[['first_instructor', 'section']]
    raw.drop_duplicates()
    raw['section'] = raw.section.astype(str) # for equal type comparisons to other data
    
    ## creating a master data frame for courses to use in optimization more efficiently
    newdata = pd.merge(core, courses, left_on = 'Section', right_on = 'Section', how = 'right')
    newdata.dropna(inplace = True)
    newdata = pd.merge(raw, newdata, left_on = 'section', right_on = 'Section', how = 'right')
    newdata['type'] = newdata['type'].map({'Not Core Course': 0, 'Core Course': 1})
    
    ## creating our penalty values for time slots to optimize upon
    ## we set the optimal class time to be 2:00 pm for ease of access for the majority of undergraduates
    ## courses are penalized for being early or later at 5 point increments per slot distance from 2:00 pm
    penalty = pd.DataFrame({'times': list(classes.index.values), 'penalty' : ([0]*len(list(classes.index.values)))})
    penalty['penalty'] = [15,10,5,0,5,10,15]* 5 # multiplied by 5 since there are 5 days
    
    ## BEGINNING GUROBI OPTIMIZATION
    mod = Model()

    ## defining data variables based on formulation, drawing upon data declared previously in the file
    S = newdata.Section
    T = list(classes.index.values)
    C = capacity.Room
    Ns = newdata.course
    Ns.index = S
    Ds = newdata.demand
    Ds.index = S
    Uc = capacity.Size
    Uc.index = C
    Is = newdata.type
    Is.index = S
    Hs = newdata.units
    Hs.index = S
    Pt = penalty.penalty
    Pt.index = T
    Po = newdata.first_instructor
    Po.index = S

    # CORE CLASSES TO BE TAKEN AT NIGHT
    CNs = newdata[['course', 'section']][list(pd.Series(newdata.type ==1).values)].drop_duplicates(keep = 'first', subset = 'course').section


    X = mod.addVars(S,C,T, vtype = GRB.BINARY)

    mod.setObjective( sum(Ds[s]*X[s,c,t] for s in S for c in C for t in T) - sum(Pt[t]*X[s,c,t] for s in S for c in C for t in T)  , sense = GRB.MAXIMIZE)

    # upper classroom capacity constraint
    for s in S:
        for c in C:
            for t in T:
                mod.addConstr(Ds[s]*X[s,c,t] <= Uc[c]*1.1)

    # one class per classroom and time constraint
    for c in C:
        for t in T:
            mod.addConstr(sum(X[s,c,t] for s in S) <= 1)

    # multiple day classes in same classroom and time
    for s in S:
        if Hs[s] >= 3:
            for c in C:
                mod.addConstr(X[s,c,'M 08:00'] - X[s,c,'W 08:00'] <= 0)
                mod.addConstr(X[s,c,'M 10:00'] - X[s,c,'W 10:00'] <= 0)
                mod.addConstr(X[s,c,'M 12:00'] - X[s,c,'W 12:00'] <= 0)
                mod.addConstr(X[s,c,'M 14:00'] - X[s,c,'W 14:00'] <= 0)
                mod.addConstr(X[s,c,'M 16:00'] - X[s,c,'W 16:00'] <= 0)
                mod.addConstr(X[s,c,'M 18:00'] - X[s,c,'W 18:00'] <= 0)
                mod.addConstr(X[s,c,'M 20:00'] - X[s,c,'W 20:00'] <= 0)
                mod.addConstr(X[s,c,'T 08:00'] - X[s,c,'Th 08:00'] <= 0)
                mod.addConstr(X[s,c,'T 10:00'] - X[s,c,'Th 10:00'] <= 0)
                mod.addConstr(X[s,c,'T 12:00'] - X[s,c,'Th 12:00'] <= 0)
                mod.addConstr(X[s,c,'T 14:00'] - X[s,c,'Th 14:00'] <= 0)
                mod.addConstr(X[s,c,'T 16:00'] - X[s,c,'Th 16:00'] <= 0)
                mod.addConstr(X[s,c,'T 18:00'] - X[s,c,'Th 18:00'] <= 0)
                mod.addConstr(X[s,c,'T 20:00'] - X[s,c,'Th 20:00'] <= 0)

                mod.addConstr(X[s,c,'F 08:00'] <= 0)
                mod.addConstr(X[s,c,'F 10:00'] <= 0)
                mod.addConstr(X[s,c,'F 12:00'] <= 0)
                mod.addConstr(X[s,c,'F 14:00'] <= 0)
                mod.addConstr(X[s,c,'F 16:00'] <= 0)
                mod.addConstr(X[s,c,'F 18:00'] <= 0)
                mod.addConstr(X[s,c,'F 20:00'] <= 0)

    # each section only has a sum of exactly one for the X variable in total for 2 unit classes
        # sum of exactly two for 4 unit classes
        # this constraint also guarantees that each of the sections has a place in the schedule
    for s in S:
        if Hs[s] <=2:
            mod.addConstr(sum(X[s,c,t] for c in C for t in T) == 1)
        elif Hs[s] >= 4:
            mod.addConstr(sum(X[s,c,t] for c in C for t in T) == 2)


    # professor constraint
    for p in Po:
        for t in T:
            mod.addConstr(sum(X[s,c,t] for c in C for s in list(Po[Po == p].index.values))   <= 1)

    # core course at night
    nightclasses = []
    for t in T:
        if re.search(r'(18:00|20:00)', t):
            nightclasses.append(t)
    nightclasses

    for s in CNs:
        mod.addConstr(sum(X[s,c,t] for c in C for t in nightclasses) >= 1)


    mod.setParam('outputflag',False)
    mod.optimize()

    mod.objVal
    
    
    ## creating a structure to place optimized values into
    output = pd.DataFrame(list(product(S,C,T)), columns = ['section', 'classroom', 'timeslot'])
    output['decision'] = 0
    
    ## reading those values in a loop
    for s in S:
        for c in C:
            for t in T:
                if X[s,c,t].x:
                    output.loc[(output.section == s) & (output.classroom == c) & (output.timeslot == t), 'decision']=1
    
    ## segmenting our data on only section placements to save computing time
    final = output.loc[output.decision == 1, :]
    
    ## re-indexing our dataframe so we can use 1->n looping 
    final.index = list(range(0, len(final)))
    ## looping through our decisions and storing them in the correct placement in the classes schedule template
    for i in range(0, len(final)):
        classes.loc[final.loc[i, 'timeslot'], final.loc[i, 'classroom']] = final.loc[i, 'section']    
        
    ## solving an instance specific data integrity issue which generalizes to future issues
    classes = classes.fillna(0)
    final_template = classes

    ## exporting finished schedule
    final_template.to_excel(output_info)
    
if __name__=='__main__':
    import sys, os
    if len(sys.argv)!= 5:
        print('Correct syntax: python optimize.py course_info room_capacities professor_info output_info')
    else:
        print (f'Running {sys.argv[0]} using argument list {sys.argv}')
        inputFile1 = sys.argv[1]
        inputFile2 = sys.argv[2]
        inputFile3 = sys.argv[3]
        if os.path.exists(inputFile1) & os.path.exists(inputFile2) & os.path.exists(inputFile3):
            optimize(inputFile1, inputFile2, inputFile3, sys.argv[4])
            print(f'Successfully optimized. Results in "{sys.argv[4]}"')
        else:
            print('One of input files not found!')  
   

    
